In [ ]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
from numpy import float64 as f64
import matplotlib as mpl
from scipy.interpolate import CubicSpline

In [ ]:
# For interactive plots
%matplotlib widget
%cd '/home/jovyan/work/labs/lab-2.2'

### Utilities

In [ ]:
label_font = 12
markersize = 8

In [ ]:
def setup_axes(ax):
    ax.get_xaxis().set_visible(True)
    ax.get_yaxis().set_visible(True)
    ax.xaxis.set_minor_locator(mpl.ticker.AutoMinorLocator())
    ax.grid(True, which="both")
    plt.rcParams.update({"figure.autolayout": True})

### Data analysis

In [ ]:
df_neon = pd.read_csv(f"data/neon.csv")
df_mercury = pd.read_csv(f"data/mercury.csv")
df_hydrogen = pd.read_csv(f"data/hydrogen.csv")
df_calibrate = pd.concat([df_mercury, df_neon]).sort_values(by=['lambda, units'])
df_iodine = pd.read_csv(f"data/iodine.csv")

In [ ]:
fit_degree = 4
x_calibrate = df_calibrate["lambda, units"]
y_calibrate = df_calibrate["lambda, A"]
xs_calibrate = np.linspace(min(x_calibrate), max(x_calibrate), 4096)
calibrate_poly = np.polyfit(x_calibrate, y_calibrate, fit_degree)

In [ ]:
def interpolate_lambda(counts: float):
    return np.polyval(calibrate_poly, counts)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 6))
setup_axes(ax)
ax.scatter(x_calibrate, y_calibrate)
ax.plot(xs_calibrate, np.vectorize(interpolate_lambda)(xs_calibrate))
ax.set_xlabel("$n$, деления")
ax.set_ylabel("$\lambda, A$")
ax.set_title("Рис. 1. Калибровочный график (шкала перевода делений в длину волны)")
fig.savefig(f"output/calibrate.pdf")

In [ ]:
df_hydrogen["lambda, A"] = np.vectorize(interpolate_lambda)(df_hydrogen["lambda, units"])

In [ ]:
print(df_hydrogen)

In [ ]:
df_iodine["lambda, A"] = np.vectorize(interpolate_lambda)(df_iodine["lamda, units"])

In [ ]:
print(df_iodine)

In [ ]:
speed_of_light = 3e8 # m/s
plank_constant = 6.626e-34 # j * s
ev_to_joules = 1.6e-19 # j

In [ ]:
lambda_1_0 = df_iodine["lambda, A"].iloc[0] * 1e-10
lambda_1_5 = df_iodine["lambda, A"].iloc[1] * 1e-10

In [ ]:
nu_1_0 = speed_of_light / lambda_1_0
nu_1_5 = speed_of_light / lambda_1_5

In [ ]:
print(plank_constant * (nu_1_5 - nu_1_0) / 5.0 / ev_to_joules)